In [17]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [18]:
X=np.array([[1,3],[15,17],[3,6],[22,24],[6,5],[7,6]])
Y=np.array([[1],[0],[1],[0],[1],[1]])
print(X.shape, Y.shape)

(6, 2) (6, 1)


In [19]:
Xt = np.tile(X,(10,1))
Yt= np.tile(Y,(10,1))   
print(Xt.shape, Yt.shape)  
X=Xt
Y=Yt

(60, 2) (60, 1)


In [20]:
print(f"Temperature Max, Min pre normalization: {np.max(X[:,0]):0.2f}, {np.min(X[:,0]):0.2f}")
print(f"Duration    Max, Min pre normalization: {np.max(X[:,1]):0.2f}, {np.min(X[:,1]):0.2f}")
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X)  # learns mean, variance
Xn = norm_l(X)
print(f"Temperature Max, Min post normalization: {np.max(Xn[:,0]):0.2f}, {np.min(Xn[:,0]):0.2f}")
print(f"Duration    Max, Min post normalization: {np.max(Xn[:,1]):0.2f}, {np.min(Xn[:,1]):0.2f}")

Temperature Max, Min pre normalization: 22.00, 1.00
Duration    Max, Min pre normalization: 24.00, 3.00
Temperature Max, Min post normalization: 1.79, -1.10
Duration    Max, Min post normalization: 1.81, -0.94


In [21]:
#WHY axis=1 or -1 is working???
#this is how we do preprocessing the data
t=np.array([1,2,3,4,5,5,5,6,6,7]).reshape(5,-1)
tt=np.array([1,2,3])
#tt=np.array([[1,1],[2,2],[3,3]])  if use this then axis=-1 using the last index for calc different mean and variance
print(t)
norm_l = tf.keras.layers.Normalization(axis=None)
norm_l.adapt(tt)
tn=norm_l(t)
print(tn)

[[1 2]
 [3 4]
 [5 5]
 [5 6]
 [6 7]]
tf.Tensor(
[[-1.2247448  0.       ]
 [ 1.2247448  2.4494896]
 [ 3.6742344  3.6742344]
 [ 3.6742344  4.898979 ]
 [ 4.898979   6.123724 ]], shape=(5, 2), dtype=float32)


In [22]:
def g(z):
  zz=np.float128(z)
  return 1/(1+np.exp(-zz))



""" overflow encountered in exp in Python.

This warning occurs while using the NumPy library’s exp() function upon using on a value that is too large.
This warning occurs because the maximum size of data that can be used in NumPy is float64 whose maximum range is 1.7976931348623157e+308. 
Upon taking logarithm its value becomes 709.782. For any larger value than this, the warning is generated.

Let us discuss ways to fix this.

Method 1: Using float128
The data type float64 can be changed to float128"""

' overflow encountered in exp in Python.\n\nThis warning occurs while using the NumPy library’s exp() function upon using on a value that is too large.\nThis warning occurs because the maximum size of data that can be used in NumPy is float64 whose maximum range is 1.7976931348623157e+308. \nUpon taking logarithm its value becomes 709.782. For any larger value than this, the warning is generated.\n\nLet us discuss ways to fix this.\n\nMethod 1: Using float128\nThe data type float64 can be changed to float128'

In [23]:
 def my_dense(a_in,W,b):
  """
    Computes dense layer
    Args:
      a_in (ndarray (n, )) : Data, 1 example 
      W    (ndarray (n,j)) : Weight matrix, n features per unit, j units
      b    (ndarray (j, )) : bias vector, j units  
    Returns
      a_out (ndarray (j,))  : j units|
    """
  units = W.shape[1]
  a_out = np.zeros(units)
  for j in range(units):               
    w = W[:,j]                                    
    z = np.dot(w, a_in) + b[j]         
    a_out[j] = g(z)               
  return(a_out)

"""Note: You can also implement the function above to accept g as an additional
 parameter (e.g. my_dense(a_in, W, b, g)). In this notebook though, you will only 
 use one type of activation function (i.e. sigmoid) so it's okay to make it constant
  and define it outside the function. That's what you did in the code above and it makes 
  the function calls in the next code cells simpler. Just keep in mind that passing it as 
  a parameter is also an acceptable implementation. You will see that in this week's assignment."""

"Note: You can also implement the function above to accept g as an additional\nparameter (e.g. my_dense(a_in, W, b, g)). In this notebook though, you will only \nuse one type of activation function (i.e. sigmoid) so it's okay to make it constant\n and define it outside the function. That's what you did in the code above and it makes \n the function calls in the next code cells simpler. Just keep in mind that passing it as \n a parameter is also an acceptable implementation. You will see that in this week's assignment."

In [24]:
# i did this, working exactly same
def dense1(a_in,W,b):
  """
    Computes dense layer
    Args:
      a_in (ndarray (n, )) : Data, 1 example 
      W    (ndarray (n,j)) : Weight matrix, n features per unit, j units
      b    (ndarray (j, )) : bias vector, j units  
    Returns
      a_out (ndarray (j,))  : j units|
    """
  units = W.shape[1]
  #m=W.shape[0]
  #a_out = np.zeros(units,m)                                        
  z = np.dot(a_in,W) + b       
  a_out = g(z)               
  return(a_out)

In [25]:
a=np.random.rand(5)
w=np.random.rand(20).reshape(5,4)
b=np.random.rand(4)

print(my_dense(a,w,b))
print(dense1(a,w,b))


[0.88216866 0.9191386  0.82391566 0.90372313]
[0.88216866 0.9191386  0.82391566 0.90372313]


In [26]:
def my_sequential(x, W1, b1, W2, b2):
    a1 = my_dense(x,  W1, b1)
    a2 = my_dense(a1, W2, b2)
    return(a2)

In [27]:
# i made this
def sequential1(X,W1,b1,W2,b2):
  a1=dense1(X,W1,b1)
  a2=dense1(a1,W2,b2)
  return a2

In [28]:
W1_tmp = np.array( [[-8.93,  0.29, 12.9 ], [-0.1,  -7.32, 10.81]] )
b1_tmp = np.array( [-9.82, -9.28,  0.96] )
W2_tmp = np.array( [[-31.18], [-27.59], [-32.56]] )
b2_tmp = np.array( [15.41] )

In [29]:
"""Once you have a trained model, you can then use it to make predictions. Recall that the output of our model is a probability. 
In this case, the probability of a good roast. To make a decision, one must apply the probability to a threshold. In this case, we will use 0.5

Let's start by writing a routine similar to Tensorflow's model.predict(). This will take a matrix 𝑋
 with all 𝑚
 examples in the rows and make a prediction by running the model."""

def my_predict(X, W1, b1, W2, b2):
  m = X.shape[0]
  p = np.zeros((m,1))
  for i in range(m):
    p[i,0] = my_sequential(X[i], W1, b1, W2, b2)
  return(p)

In [30]:
X_tst = np.array([
    [200,13.9],  # postive example
    [200,17]])   # negative example
X_tstn = norm_l(X_tst)  # remember to normalize
predictions = my_predict(X_tstn, W1_tmp, b1_tmp, W2_tmp, b2_tmp)


print(predictions)

print(sequential1(X_tst,W1_tmp, b1_tmp, W2_tmp, b2_tmp)) # mine also working exactly same

[[3.56327729e-08]
 [3.56327729e-08]]
[[3.56327729e-08]
 [3.56327729e-08]]


In [31]:
yhat = np.zeros_like(predictions)
print(yhat)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:                #in this value comes [False] so how it evaluates
        yhat[i] = 1
    else:
        yhat[i] = 0
print(f"decisions = \n{yhat}")

[[0.]
 [0.]]
decisions = 
[[0.]
 [0.]]


In [32]:
# more concise way of doing
yhat = (predictions >= 0.5).astype(int)
print(f"decisions = \n{yhat}")

decisions = 
[[0]
 [0]]


In [33]:
a=np.array([[1,2,3],[1,2,3]])
w=np.array([1,2,3])
print(np.matmul(a,w))

[14 14]


In [34]:
print(a)
print(a.T)

[[1 2 3]
 [1 2 3]]
[[1 1]
 [2 2]
 [3 3]]
